In [1]:
!apt-get install libomp-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libomp-14-dev libomp5-14
Suggested packages:
  libomp-14-doc
The following NEW packages will be installed:
  libomp-14-dev libomp-dev libomp5-14
0 upgraded, 3 newly installed, 0 to remove and 45 not upgraded.
Need to get 738 kB of archives.
After this operation, 8,991 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 libomp5-14 amd64 1:14.0.0-1ubuntu1.1 [389 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 libomp-14-dev amd64 1:14.0.0-1ubuntu1.1 [347 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libomp-dev amd64 1:14.0-55~exp2 [3,074 B]
Fetched 738 kB in 1s (1,095 kB/s)
Selecting previously unselected package libomp5-14:amd64.
(Reading database ... 121752 files and directories currently installed.)
Preparing to unpack .../libomp5-14_1%3a

In [7]:
%%writefile bubblesort.cpp
#include <iostream>
#include <vector>
#include <omp.h>

using namespace std;

void bubble_sort_serial(vector<int>& arr) {
    int n = arr.size();
    bool swapped;
    for (int i = 0; i < n-1; i++) {
        swapped = false;
        for (int j = 0; j < n-i-1; j++) {
            if (arr[j] > arr[j+1]) {
                swap(arr[j], arr[j+1]);
                swapped = true;
            }
        }
        if (!swapped) {
            break;
        }
    }
}

void bubble_sort_odd_even(vector<int>& arr) {
    bool isSorted = false;
    while (!isSorted) {
        isSorted = true;
        #pragma omp parallel for
        for (int i = 0; i < arr.size() - 1; i += 2) {
            if (arr[i] > arr[i + 1]) {
                swap(arr[i], arr[i + 1]);
                isSorted = false;
            }
        }
        #pragma omp parallel for
        for (int i = 1; i < arr.size() - 1; i += 2) {
            if (arr[i] > arr[i + 1]) {
                swap(arr[i], arr[i + 1]);
                isSorted = false;
            }
        }
    }
}

int main() {
    int n;
    cout << "Enter the number of elements: ";
    cin >> n;

    vector<int> arr(n), arr_copy(n);
    cout << "Enter the elements: ";
    for (int i = 0; i < n; i++) {
        cin >> arr[i];
        arr_copy[i] = arr[i];
    }

    double start, end;

    // Measure performance of serial bubble sort
    start = omp_get_wtime();
    bubble_sort_serial(arr);
    end = omp_get_wtime();
    cout << "Sorted array using serial bubble sort: ";
    for (int i = 0; i < n; i++) {
        cout << arr[i] << " ";
    }
    cout << endl;
    cout << "Serial bubble sort time: " << end - start << " seconds" << endl;

    // Measure performance of parallel bubble sort using odd-even transposition
    start = omp_get_wtime();
    bubble_sort_odd_even(arr_copy);
    end = omp_get_wtime();
    cout << "Sorted array using parallel bubble sort: ";
    for (int i = 0; i < n; i++) {
        cout << arr_copy[i] << " ";
    }
    cout << endl;
    cout << "Parallel bubble sort using odd-even transposition time: " << end - start << " seconds" << endl;

    return 0;
}
//g++ -fopenmp bubblesort.cpp -o bubblesort
//./bubblesort

Writing bubblesort.cpp


In [9]:
!clang++ -fopenmp bubblesort.cpp -o bubblesort
!./bubblesort

Enter the number of elements: 4
Enter the elements: 11 2 33 5
Sorted array using serial bubble sort: 2 5 11 33 
Serial bubble sort time: 4.05312e-06 seconds
Sorted array using parallel bubble sort: 2 5 11 33 
Parallel bubble sort using odd-even transposition time: 0.00133991 seconds
